## 1. Import Libraries và Khởi tạo Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import (
    col, explode, split, trim, current_timestamp, row_number, 
    when, regexp_replace, monotonically_increasing_id, dense_rank, udf, collect_list
)
from pyspark.sql.window import Window
import os

# Set AWS environment variables for MinIO
os.environ['AWS_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admin123'

# Khởi tạo Spark Session với cấu hình tối ưu cho memory
spark = (
    SparkSession.builder.appName("Load_Data_To_Gold_Tables")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "1536m")
    .config("spark.executor.cores", "2")
    .config("spark.memory.fraction", "0.6")
    .config("spark.memory.storageFraction", "0.3")
    .config("spark.sql.shuffle.partitions", "50")
    .config("spark.default.parallelism", "50")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .config("spark.shuffle.spill.compress", "true")
    .config("spark.shuffle.compress", "true")
    .config("spark.rdd.compress", "true")
    # ===== Iceberg Catalog qua Nessie =====
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://gold/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    # ===== Cấu hình MinIO =====
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    # ===== Spark + Hadoop S3 connector =====
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print("✓ Spark Session đã được khởi tạo!")
print(f"Spark Master: {spark.sparkContext.master}")
print(f"Application ID: {spark.sparkContext.applicationId}")


25/12/06 12:25:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


✓ Spark Session đã được khởi tạo!
Spark Master: spark://spark-master:7077
Application ID: app-20251206122503-0003


## 2. Đọc Dữ Liệu từ Result Model và Lọc Records Chưa Xử Lý

In [2]:
# Đọc dữ liệu từ bảng result_multi_model
df_result = spark.table("nessie.silver_tables.result_multi_model")

print(f"Tổng số records trong result_multi_model: {df_result.count()}")

# Kiểm tra xem Post table đã có dữ liệu chưa
try:
    df_existing_posts = spark.table("nessie.gold_result_model_multi_task.Post")
    existing_post_ids = [row.postID for row in df_existing_posts.select("postID").distinct().collect()]
    print(f"Tìm thấy {len(existing_post_ids)} posts đã được xử lý trước đó")
    
    # Lọc ra các posts chưa được xử lý
    if existing_post_ids:
        df_result_new = df_result.filter(~col("postID").isin(existing_post_ids))
        print(f"Số lượng posts mới cần xử lý: {df_result_new.count()}")
        
        if df_result_new.count() == 0:
            print("\n⚠️ Không có posts mới để xử lý. Tất cả dữ liệu đã được load vào Gold tables.")
            print("Nếu muốn load lại, hãy xóa dữ liệu trong Gold tables trước.")
        else:
            df_result = df_result_new
    else:
        print("Không có posts nào trong Gold tables, sẽ xử lý toàn bộ dữ liệu")
        
except Exception as e:
    print(f"Gold tables chưa có dữ liệu hoặc chưa tồn tại: {e}")
    print("Sẽ xử lý toàn bộ dữ liệu từ result_multi_model")

print("\nSchema:")
df_result.printSchema()
print("\nSample data:")
df_result.show(5, truncate=False)


Tổng số records trong result_multi_model: 3295
Tìm thấy 0 posts đã được xử lý trước đó
Không có posts nào trong Gold tables, sẽ xử lý toàn bộ dữ liệu

Schema:
root
 |-- postID: string (nullable = true)
 |-- timePublish: timestamp (nullable = true)
 |-- description_Normalized: string (nullable = true)
 |-- Label_NER: string (nullable = true)
 |-- Label_Topic: string (nullable = true)
 |-- Label_Intent: string (nullable = true)
 |-- likeCount: integer (nullable = true)
 |-- commentCount: integer (nullable = true)
 |-- shareCount: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)


Sample data:


+-----------------------------------------+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------

## 3. Transform và Load vào Table Entity

Parse Label_NER để tách các entity (B-MAJOR, I-MAJOR, B-ORG, I-ORG, etc.)

In [3]:
# Parse NER labels để extract entities
@udf(returnType=ArrayType(StructType([
    StructField("entityType", StringType()),
    StructField("entityName", StringType())
])))
def extract_entities(ner_labels, text):
    """Extract entities from NER labels and text"""
    if not ner_labels or not text:
        return []
    
    labels = ner_labels.split()
    words = text.split()
    
    if len(labels) != len(words):
        return []
    
    entities = []
    current_entity = None
    current_type = None
    current_words = []
    
    for label, word in zip(labels, words):
        if label.startswith('B-'):
            if current_entity and current_words:
                entities.append({
                    'entityType': current_type,
                    'entityName': ' '.join(current_words)
                })
            current_type = label[2:]
            current_words = [word]
            current_entity = True
        elif label.startswith('I-') and current_entity:
            if label[2:] == current_type:
                current_words.append(word)
        else:
            if current_entity and current_words:
                entities.append({
                    'entityType': current_type,
                    'entityName': ' '.join(current_words)
                })
            current_entity = None
            current_type = None
            current_words = []
    
    if current_entity and current_words:
        entities.append({
            'entityType': current_type,
            'entityName': ' '.join(current_words)
        })
    
    return entities

# Extract entities
df_entities_raw = df_result.select(
    col("postID"),
    explode(extract_entities(col("Label_NER"), col("description_Normalized"))).alias("entity")
).select(
    col("postID"),
    col("entity.entityType").alias("entityType"),
    col("entity.entityName").alias("entityName")
).filter(
    (col("entityType").isNotNull()) & (col("entityName").isNotNull())
)

print(f"\nTổng số entities được extract: {df_entities_raw.count()}")
df_entities_raw.show(10, truncate=False)

# Lấy unique entities và gán ID
df_unique_entities = df_entities_raw.select(
    col("entityType"),
    col("entityName")
).distinct()

window_spec = Window.orderBy("entityType", "entityName")
df_entities = df_unique_entities.withColumn(
    "entityID",
    row_number().over(window_spec)
).withColumn(
    "created_at", current_timestamp()
).withColumn(
    "updated_at", current_timestamp()
).select(
    col("entityID"),
    col("entityType"),
    col("entityName"),
    col("created_at"),
    col("updated_at")
)

print(f"\nSố lượng unique entities từ dữ liệu mới: {df_entities.count()}")
print("\nDistribution by entity type:")
df_entities.groupBy("entityType").count().orderBy(col("count").desc()).show()

# Kiểm tra và merge với entities đã có
print("\n=== Loading data into Entity table ===")
try:
    df_existing_entities = spark.table("nessie.gold_result_model_multi_task.Entity")
    existing_count = df_existing_entities.count()
    print(f"Tìm thấy {existing_count} entities đã tồn tại")
    
    max_entity_id = df_existing_entities.agg({"entityID": "max"}).collect()[0][0]
    if max_entity_id is None:
        max_entity_id = 0
    print(f"Max entityID hiện tại: {max_entity_id}")
    
    df_new_entities = df_entities.join(
        df_existing_entities.select("entityType", "entityName"),
        on=["entityType", "entityName"],
        how="left_anti"
    )
    
    new_count = df_new_entities.count()
    print(f"Số lượng entities mới cần thêm: {new_count}")
    
    if new_count > 0:
        window_spec = Window.orderBy("entityType", "entityName")
        df_new_entities = df_new_entities.withColumn(
            "entityID",
            row_number().over(window_spec) + max_entity_id
        ).select(
            col("entityID"),
            col("entityType"),
            col("entityName"),
            col("created_at"),
            col("updated_at")
        )
        
        df_new_entities.writeTo("nessie.gold_result_model_multi_task.Entity") \
            .using("iceberg") \
            .append()
        print(f"✓ Đã thêm {new_count} entities mới!")
        
        df_entities = df_existing_entities.union(df_new_entities)
    else:
        print("⚠️ Không có entities mới cần thêm")
        df_entities = df_existing_entities
        
except Exception as e:
    print(f"Table Entity chưa có dữ liệu hoặc chưa tồn tại: {e}")
    print("Tạo mới table với dữ liệu hiện tại")
    df_entities.writeTo("nessie.gold_result_model_multi_task.Entity") \
        .using("iceberg") \
        .create()
    print(f"✓ Đã tạo table Entity với {df_entities.count()} records!")



Tổng số entities được extract: 17876
+------------------------------------+----------+-----------------------+
|postID                              |entityType|entityName             |
+------------------------------------+----------+-----------------------+
|1922491415274632                    |ORG       |Trường Đại_học Giáo_dục|
|@vihocduoc/video/7496116925982887186|MAJ       |Ngành Dược             |
|@vihocduoc/video/7496116925982887186|MAJ       |Ngành Dược             |
|@vihocduoc/video/7496116925982887186|MAJ       |Ngành Dược             |
|@vihocduoc/video/7496116925982887186|MISC      |sinh_viên              |
|@vihocduoc/video/7496116925982887186|MAJ       |Dược                   |
|@vihocduoc/video/7496116925982887186|MAJ       |ngành Dược             |
|@vihocduoc/video/7496116925982887186|MISC      |sinhvienyduoc          |
|@vihocduoc/video/7496116925982887186|MISC      |sinhvienduoc           |
|@vihocduoc/video/7496116925982887186|MISC      |sinhvien               |



Số lượng unique entities từ dữ liệu mới: 7537

Distribution by entity type:


+----------+-----+
|entityType|count|
+----------+-----+
|       MAJ| 2593|
|       ORG| 1701|
|      MISC| 1163|
|       SCO|  449|
|      DATE|  352|
|       FEE|  308|
|       SAL|  237|
|       LOC|  180|
|      SUBJ|  168|
|        EX|  156|
|      TERM|  116|
|       PRO|  114|
+----------+-----+


=== Loading data into Entity table ===
Tìm thấy 0 entities đã tồn tại
Max entityID hiện tại: 0


Số lượng entities mới cần thêm: 7537


✓ Đã thêm 7537 entities mới!


## 4. Transform và Load vào Table Topic

In [4]:
# Parse Label_Topic (multi-label, phân cách bằng '|')
df_topics_raw = df_result.select(
    explode(split(col("Label_Topic"), "\\|")).alias("topicName")
).filter(
    (col("topicName").isNotNull()) & 
    (col("topicName") != "None") &
    (trim(col("topicName")) != "")
)

df_unique_topics = df_topics_raw.select(
    trim(col("topicName")).alias("topicName")
).distinct()

window_spec = Window.orderBy("topicName")
df_topics = df_unique_topics.withColumn(
    "topicID",
    row_number().over(window_spec)
).withColumn(
    "created_at", current_timestamp()
).withColumn(
    "updated_at", current_timestamp()
).select(
    col("topicID"),
    col("topicName"),
    col("created_at"),
    col("updated_at")
)

print(f"Số lượng unique topics từ dữ liệu mới: {df_topics.count()}")
df_topics.show(20, truncate=False)

print("\n=== Loading data into Topic table ===")
try:
    df_existing_topics = spark.table("nessie.gold_result_model_multi_task.Topic")
    existing_count = df_existing_topics.count()
    print(f"Tìm thấy {existing_count} topics đã tồn tại")
    
    max_topic_id = df_existing_topics.agg({"topicID": "max"}).collect()[0][0]
    if max_topic_id is None:
        max_topic_id = 0
    print(f"Max topicID hiện tại: {max_topic_id}")
    
    df_new_topics = df_topics.join(
        df_existing_topics.select("topicName"),
        on="topicName",
        how="left_anti"
    )
    
    new_count = df_new_topics.count()
    print(f"Số lượng topics mới cần thêm: {new_count}")
    
    if new_count > 0:
        window_spec = Window.orderBy("topicName")
        df_new_topics = df_new_topics.withColumn(
            "topicID",
            row_number().over(window_spec) + max_topic_id
        ).select(
            col("topicID"),
            col("topicName"),
            col("created_at"),
            col("updated_at")
        )
        
        df_new_topics.writeTo("nessie.gold_result_model_multi_task.Topic") \
            .using("iceberg") \
            .append()
        print(f"✓ Đã thêm {new_count} topics mới!")
        
        df_topics = df_existing_topics.union(df_new_topics)
    else:
        print("⚠️ Không có topics mới cần thêm")
        df_topics = df_existing_topics
        
except Exception as e:
    print(f"Table Topic chưa có dữ liệu hoặc chưa tồn tại: {e}")
    print("Tạo mới table với dữ liệu hiện tại")
    df_topics.writeTo("nessie.gold_result_model_multi_task.Topic") \
        .using("iceberg") \
        .create()
    print(f"✓ Đã tạo table Topic với {df_topics.count()} records!")


Số lượng unique topics từ dữ liệu mới: 10


+-------+-------------------+--------------------------+--------------------------+
|topicID|topicName          |created_at                |updated_at                |
+-------+-------------------+--------------------------+--------------------------+
|1      |CAREER             |2025-12-06 12:26:05.241372|2025-12-06 12:26:05.241372|
|2      |CERTIFICATE        |2025-12-06 12:26:05.241372|2025-12-06 12:26:05.241372|
|3      |LANGUAGE           |2025-12-06 12:26:05.241372|2025-12-06 12:26:05.241372|
|4      |MAJOR              |2025-12-06 12:26:05.241372|2025-12-06 12:26:05.241372|
|5      |OTHER              |2025-12-06 12:26:05.241372|2025-12-06 12:26:05.241372|
|6      |STUDENT_LIFE       |2025-12-06 12:26:05.241372|2025-12-06 12:26:05.241372|
|7      |STUDY              |2025-12-06 12:26:05.241372|2025-12-06 12:26:05.241372|
|8      |SUBJECT_COMBINATION|2025-12-06 12:26:05.241372|2025-12-06 12:26:05.241372|
|9      |TUITION            |2025-12-06 12:26:05.241372|2025-12-06 12:26:05.

Số lượng topics mới cần thêm: 10


✓ Đã thêm 10 topics mới!


## 5. Transform và Load vào Table Post

In [5]:
# Tạo table Post từ result model
df_posts = df_result.select(
    col("postID"),
    col("description_Normalized").alias("description"),
    col("timePublish"),
    col("likeCount"),
    col("commentCount"),
    col("shareCount"),
    col("Label_Intent").alias("intent"),
    col("type")
).withColumn(
    "created_at", current_timestamp()
).withColumn(
    "updated_at", current_timestamp()
)

total_posts = df_posts.count()
print(f"Tổng số posts mới: {total_posts}")
df_posts.show(5, truncate=80)

# Load vào table Post với batch processing để tránh OOM
print("\n=== Loading data into Post table ===")
if total_posts > 0:
    # Repartition để tối ưu memory
    df_posts = df_posts.repartition(20)
    
    try:
        # Batch processing: Xử lý 1000 records mỗi lần
        BATCH_SIZE = 1000
        num_batches = (total_posts + BATCH_SIZE - 1) // BATCH_SIZE
        
        if num_batches > 1:
            print(f"Xử lý {num_batches} batches (mỗi batch ~{BATCH_SIZE} records)...")
            
            # Lấy danh sách postID để chia batch
            post_ids = [row.postID for row in df_posts.select("postID").collect()]
            
            for i in range(num_batches):
                start_idx = i * BATCH_SIZE
                end_idx = min((i + 1) * BATCH_SIZE, total_posts)
                batch_ids = post_ids[start_idx:end_idx]
                
                print(f"  Batch {i+1}/{num_batches}: {len(batch_ids)} records...", end=" ")
                
                # Filter batch
                df_batch = df_posts.filter(col("postID").isin(batch_ids))
                
                # Write batch
                if i == 0:
                    try:
                        df_batch.writeTo("nessie.gold_result_model_multi_task.Post") \
                            .using("iceberg") \
                            .append()
                        print("✓")
                    except Exception as e:
                        if "table does not exist" in str(e).lower() or "not found" in str(e).lower():
                            df_batch.writeTo("nessie.gold_result_model_multi_task.Post") \
                                .using("iceberg") \
                                .create()
                            print("✓ (created)")
                        else:
                            raise e
                else:
                    df_batch.writeTo("nessie.gold_result_model_multi_task.Post") \
                        .using("iceberg") \
                        .append()
                    print("✓")
            
            print(f"\n✓ Đã thêm tổng cộng {total_posts} posts!")
        else:
            # Ít hơn 1000 records, write trực tiếp
            print("Số lượng nhỏ, write trực tiếp...")
            try:
                df_posts.writeTo("nessie.gold_result_model_multi_task.Post") \
                    .using("iceberg") \
                    .append()
                print(f"✓ Đã thêm {total_posts} posts!")
            except Exception as e:
                if "table does not exist" in str(e).lower() or "not found" in str(e).lower():
                    df_posts.writeTo("nessie.gold_result_model_multi_task.Post") \
                        .using("iceberg") \
                        .create()
                    print(f"✓ Đã tạo table Post với {total_posts} records!")
                else:
                    raise e
                    
    except Exception as e:
        print(f"\n✗ Lỗi: {e}")
        raise e
else:
    print("⚠️ Không có posts mới cần thêm")


Tổng số posts mới: 3295
+-----------------------------------------+--------------------------------------------------------------------------------+-------------------+---------+------------+----------+----------+--------+--------------------------+--------------------------+
|                                   postID|                                                                     description|        timePublish|likeCount|commentCount|shareCount|    intent|    type|                created_at|                updated_at|
+-----------------------------------------+--------------------------------------------------------------------------------+-------------------+---------+------------+----------+----------+--------+--------------------------+--------------------------+
|                         1922491415274632|                                                       Trường Đại_học Giáo_dục .|2025-04-22 18:12:00|      242|         253|         8|  ask_info|facebook|2025-12-06 12:26:16

  Batch 1/4: 1000 records... 

✓
  Batch 2/4: 1000 records... 

✓
  Batch 3/4: 1000 records... 

✓
  Batch 4/4: 295 records... 

✓

✓ Đã thêm tổng cộng 3295 posts!


## 6. Transform và Load vào Table Post_Entity

Bảng quan hệ M:N giữa Post và Entity

In [6]:
# Extract entities với postID và order
df_post_entities_raw = df_result.select(
    col("postID"),
    explode(extract_entities(col("Label_NER"), col("description_Normalized"))).alias("entity")
).select(
    col("postID"),
    col("entity.entityType").alias("entityType"),
    col("entity.entityName").alias("entityName")
).filter(
    (col("entityType").isNotNull()) & (col("entityName").isNotNull())
)

# Add entity order
window_spec = Window.partitionBy("postID").orderBy("entityType", "entityName")
df_post_entities_ordered = df_post_entities_raw.withColumn(
    "entityOrder",
    row_number().over(window_spec)
)

# Join với Entity table để lấy entityID
df_post_entity = df_post_entities_ordered.join(
    df_entities.select("entityID", "entityType", "entityName"),
    on=["entityType", "entityName"],
    how="inner"
).select(
    col("postID"),
    col("entityID"),
    col("entityOrder"),
    col("entityName")
).withColumn(
    "created_at", current_timestamp()
).withColumn(
    "updated_at", current_timestamp()
)

print(f"Tổng số quan hệ Post-Entity mới: {df_post_entity.count()}")
df_post_entity.show(10, truncate=False)

print("\n=== Loading data into Post_Entity table ===")
if df_post_entity.count() > 0:
    try:
        df_post_entity.writeTo("nessie.gold_result_model_multi_task.Post_Entity") \
            .using("iceberg") \
            .append()
        print(f"✓ Đã thêm {df_post_entity.count()} quan hệ Post-Entity mới!")
    except Exception as e:
        if "table does not exist" in str(e).lower() or "not found" in str(e).lower():
            print("Table Post_Entity chưa tồn tại, tạo mới...")
            df_post_entity.writeTo("nessie.gold_result_model_multi_task.Post_Entity") \
                .using("iceberg") \
                .create()
            print(f"✓ Đã tạo table Post_Entity với {df_post_entity.count()} records!")
        else:
            raise e
else:
    print("⚠️ Không có quan hệ Post-Entity mới cần thêm")


Tổng số quan hệ Post-Entity mới: 17876


+----------------+--------+-----------+-------------------------------------------+--------------------------+--------------------------+
|postID          |entityID|entityOrder|entityName                                 |created_at                |updated_at                |
+----------------+--------+-----------+-------------------------------------------+--------------------------+--------------------------+
|1753055978884844|3236    |1          |ngành thiết_kế đồ_hoạ                      |2025-12-06 12:27:42.720917|2025-12-06 12:27:42.720917|
|1753084148882027|1926    |1          |Ngôn_ngữ Anh                               |2025-12-06 12:27:42.720917|2025-12-06 12:27:42.720917|
|1753084148882027|6145    |2          |Đại_học Bách_khoa Hà_Nội                   |2025-12-06 12:27:42.720917|2025-12-06 12:27:42.720917|
|1753238878866554|1791    |1          |Ngành Ngôn_ngữ Đức                         |2025-12-06 12:27:42.720917|2025-12-06 12:27:42.720917|
|1753249255532183|1909    |1      

[Stage 122:===================================================> (165 + 6) / 171]

✓ Đã thêm 17876 quan hệ Post-Entity mới!


## 7. Transform và Load vào Table Post_Topic

Bảng quan hệ M:N giữa Post và Topic

In [7]:
# Parse topics cho mỗi post
df_post_topics_raw = df_result.select(
    col("postID"),
    explode(split(col("Label_Topic"), "\\|")).alias("topicName")
).filter(
    (col("topicName").isNotNull()) & 
    (col("topicName") != "None") &
    (trim(col("topicName")) != "")
).select(
    col("postID"),
    trim(col("topicName")).alias("topicName")
)

# Join với Topic table để lấy topicID
df_post_topic = df_post_topics_raw.join(
    df_topics.select("topicID", "topicName"),
    on="topicName",
    how="inner"
).select(
    col("postID"),
    col("topicID")
).withColumn(
    "created_at", current_timestamp()
).withColumn(
    "updated_at", current_timestamp()
)

print(f"Tổng số quan hệ Post-Topic mới: {df_post_topic.count()}")
df_post_topic.show(10, truncate=False)

print("\n=== Loading data into Post_Topic table ===")
if df_post_topic.count() > 0:
    try:
        df_post_topic.writeTo("nessie.gold_result_model_multi_task.Post_Topic") \
            .using("iceberg") \
            .append()
        print(f"✓ Đã thêm {df_post_topic.count()} quan hệ Post-Topic mới!")
    except Exception as e:
        if "table does not exist" in str(e).lower() or "not found" in str(e).lower():
            print("Table Post_Topic chưa tồn tại, tạo mới...")
            df_post_topic.writeTo("nessie.gold_result_model_multi_task.Post_Topic") \
                .using("iceberg") \
                .create()
            print(f"✓ Đã tạo table Post_Topic với {df_post_topic.count()} records!")
        else:
            raise e
else:
    print("⚠️ Không có quan hệ Post-Topic mới cần thêm")


Tổng số quan hệ Post-Topic mới: 5023


+-----------------------------------------+-------+--------------------------+--------------------------+
|postID                                   |topicID|created_at                |updated_at                |
+-----------------------------------------+-------+--------------------------+--------------------------+
|1922491415274632                         |10     |2025-12-06 12:28:34.639945|2025-12-06 12:28:34.639945|
|@vihocduoc/video/7496116925982887186     |1      |2025-12-06 12:28:34.639945|2025-12-06 12:28:34.639945|
|@vihocduoc/video/7496116925982887186     |4      |2025-12-06 12:28:34.639945|2025-12-06 12:28:34.639945|
|@soctruyenthong/video/7495940840813088018|4      |2025-12-06 12:28:34.639945|2025-12-06 12:28:34.639945|
|@soctruyenthong/video/7495940840813088018|9      |2025-12-06 12:28:34.639945|2025-12-06 12:28:34.639945|
|@soctruyenthong/video/7495940840813088018|10     |2025-12-06 12:28:34.639945|2025-12-06 12:28:34.639945|
|1922682591922181                         |4  

[Stage 148:===============================================>     (154 + 6) / 171]

✓ Đã thêm 5023 quan hệ Post-Topic mới!


## 8. Verify Dữ Liệu Đã Load

In [8]:
print("="*80)
print("SUMMARY - Data Loaded to Gold Tables")
print("="*80)

tables = [
    "Entity",
    "Topic",
    "Post",
    "Post_Entity",
    "Post_Topic"
]

for table_name in tables:
    df_verify = spark.table(f"nessie.gold_result_model_multi_task.{table_name}")
    count = df_verify.count()
    print(f"\n{'='*80}")
    print(f"Table: {table_name}")
    print(f"Total records: {count}")
    print(f"{'='*80}")
    df_verify.show(5, truncate=False)

print("\n" + "="*80)
print("✓ Hoàn thành load dữ liệu vào tất cả Gold tables!")
print("="*80)

SUMMARY - Data Loaded to Gold Tables

Table: Entity
Total records: 7537
+--------+----------+-------------+--------------------------+--------------------------+
|entityID|entityType|entityName   |created_at                |updated_at                |
+--------+----------+-------------+--------------------------+--------------------------+
|1       |DATE      |01/08 – 14/08|2025-12-06 12:25:53.552424|2025-12-06 12:25:53.552424|
|2       |DATE      |01/09 đến    |2025-12-06 12:25:53.552424|2025-12-06 12:25:53.552424|
|3       |DATE      |01/09 – 12/09|2025-12-06 12:25:53.552424|2025-12-06 12:25:53.552424|
|4       |DATE      |01/2025      |2025-12-06 12:25:53.552424|2025-12-06 12:25:53.552424|
|5       |DATE      |03 - 04/9    |2025-12-06 12:25:53.552424|2025-12-06 12:25:53.552424|
+--------+----------+-------------+--------------------------+--------------------------+
only showing top 5 rows


Table: Topic
Total records: 10
+-------+-----------+--------------------------+-------------

## 9. Thống Kê và Phân Tích

In [9]:
# Thống kê số lượng entities theo type
print("\n=== Entity Distribution by Type ===")
spark.sql("""
    SELECT entityType, COUNT(*) as count
    FROM nessie.gold_result_model_multi_task.Entity
    GROUP BY entityType
    ORDER BY count DESC
""").show(truncate=False)

# Thống kê số lượng topics
print("\n=== Topic Distribution ===")
spark.sql("""
    SELECT topicName, COUNT(pt.postID) as post_count
    FROM nessie.gold_result_model_multi_task.Topic t
    LEFT JOIN nessie.gold_result_model_multi_task.Post_Topic pt ON t.topicID = pt.topicID
    GROUP BY topicName
    ORDER BY post_count DESC
""").show(20, truncate=False)

# Thống kê intent distribution
print("\n=== Intent Distribution ===")
spark.sql("""
    SELECT intent, COUNT(*) as count
    FROM nessie.gold_result_model_multi_task.Post
    GROUP BY intent
    ORDER BY count DESC
""").show(truncate=False)

# Top entities được nhắc đến nhiều nhất
print("\n=== Top 10 Most Mentioned Entities ===")
spark.sql("""
    SELECT e.entityName, e.entityType, COUNT(pe.postID) as mention_count
    FROM nessie.gold_result_model_multi_task.Entity e
    JOIN nessie.gold_result_model_multi_task.Post_Entity pe ON e.entityID = pe.entityID
    GROUP BY e.entityName, e.entityType
    ORDER BY mention_count DESC
    LIMIT 10
""").show(truncate=False)


=== Entity Distribution by Type ===
+----------+-----+
|entityType|count|
+----------+-----+
|MAJ       |2593 |
|ORG       |1701 |
|MISC      |1163 |
|SCO       |449  |
|DATE      |352  |
|FEE       |308  |
|SAL       |237  |
|LOC       |180  |
|SUBJ      |168  |
|EX        |156  |
|TERM      |116  |
|PRO       |114  |
+----------+-----+


=== Topic Distribution ===


+-------------------+----------+
|topicName          |post_count|
+-------------------+----------+
|MAJOR              |1787      |
|UNIVERSITY         |1354      |
|CAREER             |428       |
|STUDY              |420       |
|TUITION            |374       |
|OTHER              |241       |
|SUBJECT_COMBINATION|239       |
|CERTIFICATE        |74        |
|LANGUAGE           |69        |
|STUDENT_LIFE       |37        |
+-------------------+----------+


=== Intent Distribution ===


+----------------+-----+
|intent          |count|
+----------------+-----+
|share_info      |1577 |
|ask_info        |879  |
|ask_advice      |401  |
|other           |182  |
|ask_confirmation|108  |
|ask_comparison  |74   |
|ask_experience  |74   |
+----------------+-----+


=== Top 10 Most Mentioned Entities ===
+--------------+----------+-------------+
|entityName    |entityType|mention_count|
+--------------+----------+-------------+
|2025          |DATE      |329          |
|học_phí       |FEE       |326          |
|sinh_viên     |MISC      |265          |
|điểm_chuẩn    |MISC      |179          |
|học_bổng      |MISC      |164          |
|đại_học       |MISC      |135          |
|Việt_Nam      |LOC       |115          |
|năm 2025      |DATE      |111          |
|trường đại_học|MISC      |106          |
|Điểm_chuẩn    |MISC      |100          |
+--------------+----------+-------------+



## 10. Dừng Spark Session

In [10]:
# Dừng Spark Session
spark.stop()
print("✓ Spark Session đã được dừng!")


✓ Spark Session đã được dừng!
